# Homework 1 : Unix Basics, Data Access, TSCC


1. [Git and Github](3_gitgoing.ipynb#) 
    - ~~To turn in: Nothing, but you have to do it to proceed~~
    - To turn in: Link to your forked `cheatsheets` repository
2. [Jupyter Notebook Setup](4_jupyter_setup.ipynb) 
    - To turn in: Nothing, but you have to do it to proceed
    - This has you clone the github repo for the class which is necessary for the next steps
3. [Fork and clone the BIOM262 GitHub repository, create `week01` branch](5_fork_clone_biom262-2016_repo.ipynb)
    - To turn in: Nothing, but you have to do it to proceed
4. [Which promoters does NFKB bind in the genome?](7_tf_binding_promoters.ipynb)
    - To turn in: Jupyter notebook of `7_tf_binding_promoters.ipynb` submitted as a Pull Request (PR) to the `biom262/biom262-2016` repository.
5. [Bringing it all together](8_bringing_it_all_together.ipynb)
    - To turn in: Link to a GitHub repository
    
## How to turn in your homework

1. On the GitHub website for your fork of the `biom262-2016` repo, click "New Pull Request":

![](images/new_pull_request.png "New Pull Request")

2. On the "Comparing changes" page, choose "compare: week01"

![](images/comparing_changes_branches.png "Choose a branch for comparing changes")

3. Click "Create Pull Request"

![](images/create_pull_request.png)

4. In the text of the pull request, add the following text, replacing the URLs with the relevant ones for the different sections.

```
- #1: [My cheatsheets repo](http://github.com/yourgithubusername/cheatsheets)
- #5: [Our TSCC script repo])(http://github.com/githubusername/repo_from_bringing_it_all_together)
```

## FAQ and Tips and Tricks

### Logging in off-campus

For some reason TSCC isn't letting some people log on off-campus. To do that, you'll need to set up a VPN client following [these](https://blink.ucsd.edu/technology/network/connections/off-campus/VPN/index.html) instructions for the **VPN AnyConnect client**. Don't do EasyConnect because that will only send the traffic in your browser through UCSD, but you need all of the internetz through UCSD to be able to `ssh` and stuff.

### What's up with this "home" directory anyway?

There's been some confusion about what is a "home" directory. On TSCC, there is a folder called `/home`, but this is *not* **your** home directory. If you look at `/home`:

```
$ ls /home
amuratov   ecwheele       henrikse  jnussbac    r4he          ucsd-train15  ysui
areinke    elvannostrand  hocks     jpg         rit001        ucsd-train17  yuh155
b1lake     escott         ihchu     kstandis    rmande        ucsd-train43  yul277
b2dong     esticca        j6yao     mhunt       serein        ucsd-train46  yul549
bcopeland  gmccord        jbrooks   mrierari    shsathe       ux451890      z4deng
chz022     gpratt         jenhan    nbutcher    tiliu         w6ye          z9wang
cnnguyen   grbala         jhayes    obotvinnik  tkc004        wbrandler     zhz004
davids4    grosanio       jic198    ongsp       ucsd-train01  yeo-lab       zix009
```

You'll see a bunch of usernames. The idea is that everyone has their own "house," so your "house" is `/home/ucds-train##`, and this is called your "home directory."

### Ahhh I did something I didn't want to do and want to get out!!

If you ran a program or command that you didn't want to, "Ctrl-`C`" is your friend. This will exit whatever program you're in and get you back to the command line. You can think of it as a "AHH I don't know what happened and I'm scared please let me out!! Panic panic!" button.

### Add shortcuts to make logging in easier

To help you log in to TSCC, for Mac: edit your `~/.bash_profile` (on Linux, edit your ``~/.bashrc`)

```
nano ~/.bash_profile
```

Add the lines below, replacing `ucsd-train##` with your train number.

```
alias tscc='ssh ucsd-train##@tscc-login#.sdsc.edu'
alias tunneltscc='ssh -NL ####:localhost:#### ucsd-train##@tscc-login#.sdsc.edu'
```

### Still getting asked for a password?

This seems to be a problem exclusive to Mac OS X "El Capitan" users. It turns out that `ssh-add` is not permanent and you have to do it every time you want to log in. To avoid having to explicitly write that command, instead we'll edit the `~/.bash_profile` on your laptop.

On your laptop, open up `~/.bash_profile` to edit using `nano`:

```
nano ~/.bash_profile
```

Now, add the following line:

```
ssh-add ~/.ssh/biom262_rsa.pri
```

Keep in mind that the file `~/.ssh/biom262_rsa.pri` must exist! If not, then you need to re-do [setting up your logins](http://nbviewer.ipython.org/github/biom262/biom262-2016/blob/master/weeks/week01/2_tscc_setup.ipynb#Set-up-your-logins).

### Anaconda installation issues

#### Already have python error
If you're getting an error that you already have Python around that looks like this:

```
WARNING:
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Anaconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Anaconda3: /home/ucsd-train46/anaconda3
Do you wish the installer to prepend the Anaconda3 install location
to PATH in your /home/ucsd-train46/.bashrc ? [yes|no]
[no] >>> 
```
Type "`yes`"! Do not press "enter" because then it will do the default which is "`no`". If you pressed "enter", that's okay, move on to the next fix

#### Make sure the Anaconda installer *actually* added lines to your `~/.bashrc`

Use `nano` to look at your `~/.bashrc` and edit it just in case:

```
$ nano ~/.bashrc
```

If there is no line that looks like the one below (replacing `##` with your train number), then add it to the `~/.bashrc` file.

```
# added by Anaconda3 2.4.1 installer
export PATH="/home/ucsd-train##/anaconda3/bin:$PATH"
```

Save and close the file.

#### "Cannot connect to https://conda.anaconda.org/"

If you're having issues connecting to the `https://conda.anaconda.org/r` channel, do the `conda install` command with everything except the `https://conda.anaconda.org/`, as below:

```
conda install --channel r r r-irkernel seaborn
```


### Jupyter problems

#### Output after `jupyter notebook`

You will see a number get output from the `jupyter notebook --no-browser --port #### &` command before you see the rest. That is totally normal and expected.

```
$ jupyter notebook --no-browser --port #### &
[1] 12583
(biom262)[ucsd-train12@tscc-login2 ~]$ [I 13:23:05.786 NotebookApp] Writing notebook server cookie secret to /home/ucsd-train12/.local/share/jupyter/runtime/notebook_cookie_secret
[I 13:23:06.291 NotebookApp] Serving notebooks from local directory: /home/ucsd-train12
[I 13:23:06.291 NotebookApp] 0 active kernels
[I 13:23:06.291 NotebookApp] The IPython Notebook is running at: http://localhost:7788/
[I 13:23:06.291 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
```

#### Bug in instructions - you must use `--no-browser` flag

At some point the instructions on the website didn't have the `--no-browser` flag for `jupyter`. Make sure that your jupyter command includes this, and thus looks like:

```
jupyter notebook --no-browser --port #### &
```

#### "The port #### is already in use, try another random port"

This can happen if you ran `jupyter notebook` and killed it but wanted to run it again. This is because even though the `jupyter notebook` process died, there's still a "zombie" (real computer term) process running. To kill it, do:

```
$ ps
  PID TTY          TIME CMD
 6593 pts/44   00:00:00 bash
12583 pts/44   00:00:02 jupyter-noteboo
12592 pts/44   00:00:00 links
14198 pts/44   00:00:01 jupyter-noteboo
14208 pts/44   00:00:00 ps
```

Look at the number that's at the beginning of the line for `jupyter-notebook` and then use `kill -9` to stop them ("`kill`" stops the process and there's flags of the extremeness of the killing, where "`-0`" is pretty chill and "`-9`" means the meanest form like 1st degree premeditated murder):

```
$ kill -9 12583
$ kill -9 14198
```

### Tunneling: asks about the fingerprint

This process is saying it's not sure about the computer you're connecting to, which may happen because before you logged on to `tscc.sdsc.edu` and now you're using `tscc-login#.sdsc.edu`, and this is specifying a different computer and now your laptop is being super paranoid and double checking with you again.

To fix this, remove the "`&`" (called an "ampersand" or "and symbol") from the tunneling command. Thus, your tunneling command should look like:

```
ssh -NL ####:localhost:#### username@tscc-login#.sdsc.edu
```

### Tunneling: Windows

To get Jupyter notebook on your computer, you'll need to set up a "SSH Tunnel" that "listens" to that particular port. Follow [these](http://realprogrammers.com/how_to/set_up_an_ssh_tunnel_with_putty.html) instructions using the `####` port number that you created before. If you're having issues, please check in on the [gitter](https://gitter.im/biom262/biom262-2016) for help.